In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from PIL import Image, ImageDraw

import keras
from keras.models import load_model

import h5py

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


#### load photo convert into gray levels

In [2]:
filename="Sample4"
img=Image.open(filename+'.jpg')
print(img)

gray = img.convert('L')  # making it gray
# gray.save(filename+"_gray.jpg")

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4160x3120 at 0x7FAFED411748>


#### crop

In [3]:
print("original size:",gray.size)
xc=int(gray.size[0]/2)
yc=int(gray.size[1]/2)
hw=min(xc,yc)
gray=gray.crop((xc-hw, yc-hw, xc+hw, yc+hw))
print("cropped size:", gray.size)

original size: (4160, 3120)
cropped size: (3120, 3120)


#### resize

In [4]:
size=(1024,1024)
gray.thumbnail(size)
# print("resized to:", gray.size)
# gray.save(filename+'_small.jpg')

#### normalize image

In [5]:
npimage = np.array(gray)
print('min:',npimage.min(),'max:',npimage.max(),'mean:',npimage.mean())

def normalize(x,xmin,xmax):
    x=x-xmin
    return x/(xmax-xmin)*255

def bw(x):
    return np.where(x < 100, 0, 1)

data=bw(normalize(npimage,npimage.min(),npimage.max()))

min: 4 max: 189 mean: 147.97545337677002


#### remove "small" features

In [6]:
mask_data=data.copy()
frame_data=None
current_figure_size=0
edge_points=[]
sur=[(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]

def resolve_points():
    global current_figure_size
    (x,y)=edge_points.pop()
    mask_data[x,y]=200
    current_figure_size+=1
    x=min(x,1022)
    x=max(x,1)
    y=min(y,1022)
    
    for s in sur:
        cx=x+s[0]
        cy=y+s[1]
        if mask_data[cx,cy]==0:
            mask_data[cx,cy]=200
            edge_points.append((cx,cy))
    
    return len(edge_points)
            
for i in range(1024):
#     print('row',i)
    for j in range(1024):
        if mask_data[i,j]==0:
            edge_points.append((i,j))
            while resolve_points():
                pass
                
#             print("figure size:", current_figure_size)
            if current_figure_size<150:
#                 print("skipping")
                mask_data = np.where(mask_data == 200, 1, mask_data)
            elif current_figure_size>5000:
                frame_data = np.where(mask_data == 200, 1, 0)
                mask_data = np.where(mask_data == 200, 1, mask_data)
            else:
                mask_data = np.where(mask_data == 200, 250, mask_data)
            current_figure_size=0
    
data=np.where(mask_data==250,0,1)

#### save final image

In [7]:
# im = Image.fromarray(np.uint8(frame_data*254))
# im.save(filename+'_frame.jpg')

#### move 45 degree line starting from corners. Stop when sum of black pixels>5.

In [8]:
ndata=frame_data
#np.where(data == 1, 0, 1)

def get_corner(cor):
    for i in range(400):
        sum=0
        for j in range(i):
            x=i-j
            y=j
            if cor==3:
                x=1023-(i-j)
            elif cor==2:
                x=1023-(i-j)
                y=1023-j
            elif cor==1:
                y=1023-j
            sum+=ndata[x,y]
        if sum>4:
#             print(i, sum)
            pos=[]
            for j in range(i):
                x=i-j
                y=j
                if cor==3:
                    x=1023-(i-j)
                elif cor==2:
                    x=1023-(i-j)
                    y=1023-j
                elif cor==1:
                    y=1023-j
                if ndata[x,y]: pos.append((y,x))
#             print(pos)
            return pos[int(len(pos)/2)]
corners=[]        
for c in range(4):
    corners.append(get_corner(c))
print(corners)
corners.append(corners[0])

[(107, 109), (973, 65), (969, 942), (88, 927)]


In [9]:
# draw = ImageDraw.Draw(im)
# draw.line(corners, fill=128, width=3)
# draw.text(corners[0],'ZERO')
# draw.text(corners[1],'ONE')
# draw.text(corners[2],'TWO')
# draw.text(corners[3],'THREE')
# im.save(filename+'_small_BW_borders.jpg')

#### remap

In [10]:
rsize = 900
remapped = np.zeros((rsize, rsize))

def interp(p1, p2, p):
    return (p1[0]+(p2[0]-p1[0])*p, p1[1]+(p2[1]-p1[1])*p)

for x in range(rsize):
    for y in range(rsize):
        xp=x/rsize
        yp=y/rsize
        p1=interp(corners[0],corners[1],xp)
        p2=interp(corners[3],corners[2],xp)
        p3=interp(corners[1],corners[2],yp)
        p4=interp(corners[0],corners[3],yp)
        xmu=(p1[0]*p2[1]-p1[1]*p2[0])*(p3[0]-p4[0])-(p1[0]-p2[0])*(p3[0]*p4[1]-p3[1]*p4[0])
        ymu=(p1[0]*p2[1]-p1[1]*p2[0])*(p3[1]-p4[1])-(p1[1]-p2[1])*(p3[0]*p4[1]-p3[1]*p4[0])
        md=(p1[0]-p2[0])*(p3[1]-p4[1])-(p1[1]-p2[1])*(p3[0]-p4[0])
        xm=int(xmu/md)
        ym=int(ymu/md)
#         if x==0: 
#             print( 'xp:',xp,'yp:',yp, p1,p2,p3,p4)
#             print( 'xm:',xm,'ym:',ym)
        remapped[x,rsize-1-y]=data[ym,xm]
    
im = Image.fromarray(np.uint8(remapped*254))
im.save(filename+'_fixed.bmp')

#### load trained CNN

In [13]:
model = load_model('trained_model.h5')

ims_to_train=[]

def get_number(im):

    data= np.array(im)
    data = np.where(data>128,0,254)

    im = Image.fromarray(np.uint8(data))
    size=(28,28)
    im.thumbnail(size)
    # im.save(filename+'_0_1_rec.jpg')

    data= np.array(im)/254
    # print(data)
    
    ims_to_train.append(data)

    data=np.expand_dims(data, axis=0)
    data=np.expand_dims(data, axis=3)
    
#     print(data.shape)
#     print (data[0][:][:][:])

    res=model.predict(data, batch_size=1)

    print(res)
    return res.argmax()

#### cut out fields

In [14]:
result=np.zeros((9,9))
pad=10
for i in range(9):
    for j in range(9):
        fi=remapped[i*100+pad:(i+1)*100-pad,j*100+pad:(j+1)*100-pad]
        
        bl=6400-fi.sum()
        if bl>50:
            print(i,j,bl)
            result[i,j]=-1
            im = Image.fromarray(np.uint8(fi*254))
            im.save(filename+'_'+str(i)+'_'+str(j)+'_field.jpg')
            result[i,j]=get_number(im)
print(result)

0 1 414.0
[[1.0229689e-04 1.4755040e-05 9.9628985e-01 1.4172717e-03 9.2428201e-07
  5.0629092e-07 2.8319330e-06 1.4922151e-03 5.4856378e-04 1.3083810e-04]]
0 5 458.0
[[2.3194377e-03 9.7543016e-06 2.8489251e-02 2.7470833e-01 4.6489662e-05
  1.7646883e-03 1.0162210e-02 1.4575015e-04 6.6085601e-01 2.1498101e-02]]
0 7 312.0
[[9.8108291e-04 1.4138269e-05 1.3643785e-02 9.4069564e-01 1.6322707e-05
  2.2139186e-04 6.3959965e-06 1.6943149e-02 1.3317035e-03 2.6146455e-02]]
1 5 370.0
[[6.7017645e-06 7.6780880e-06 1.7126457e-01 8.0200040e-01 6.0816376e-07
  1.4055945e-06 1.0848094e-06 2.4833579e-02 1.5124894e-03 3.7152099e-04]]
1 6 270.0
[[1.6134379e-03 7.7223748e-02 6.0216922e-02 5.0788182e-01 4.9728863e-03
  1.2031189e-03 2.3210172e-04 3.1911740e-01 3.3554053e-03 2.4183221e-02]]
2 0 421.0
[[2.1713525e-03 8.0092446e-05 2.0264884e-02 4.4137418e-02 1.0027888e-04
  6.9302881e-01 1.7064260e-04 2.4924600e-02 3.9598611e-03 2.1116216e-01]]
2 2 500.0
[[7.1393284e-03 6.7321729e-05 4.4058926e-02 7.4407595e

In [17]:
ims_to_train=np.expand_dims(ims_to_train, axis=3)
ims_to_train[0,:,:,0]=2
h5f = h5py.File('digits.h5', 'w')
h5f.create_dataset('fig1', data=ims_to_train)
h5f.close()